In [1]:
from lion_core.session.branch import Branch
from lion_service import iModel

In [2]:
gpt4o = iModel(
    provider="openai",
    api_key="OPENAI_API_KEY",
    task="chat",
    model='gpt-4o',
    interval_tokens=1_000_000,
    interval_requests=10_000,
    max_tokens=4_000,
    temperature=0.1,
)

In [3]:
from pydantic import BaseModel, Field
from lion_core.session.branch import Branch

class SelectionModel(BaseModel):    
    selected: list = Field([], description="select from [positive, negative, neutral]")

branch = Branch(
    operative_model=SelectionModel,
    imodel=gpt4o,
)

a = await branch.operate(
    instruction="Answer the philosophical question provided",
    guidance="Reply concisely and informatively",
    context=["What is the meaning of life?"],
    reason=True,
    temperature=0.3
)

In [4]:
from lionfuncs import as_readable_json
print(as_readable_json(a))

{
    "selected": [],
    "title": "Exploring the Meaning of Life",
    "content": "The meaning of life is a deeply philosophical question that has been contemplated by thinkers, theologians, and scientists for centuries. Various perspectives offer different interpretations: some suggest that the meaning of life is to seek happiness, others propose it is to pursue knowledge or to fulfill a divine purpose. Existentialists argue that life has no inherent meaning, and it is up to individuals to create their own purpose. Ultimately, the meaning of life may be subjective and vary from person to person, depending on their beliefs, experiences, and values.",
    "reason": {
        "title": "Philosophical Perspectives on Life's Meaning",
        "content": "The response provides a concise overview of the diverse philosophical perspectives on the meaning of life, acknowledging that it is a complex and subjective question without a definitive answer.",
        "confidence_score": 0.95
    }
}


In [5]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-19T18:22:58.549724,Instruction,user,{'guidance': 'Reply concisely and informativel...,ln1c4abf0802a30a2-491-1ad-eba8-196f644c362,user,lnfe7143be66ee817-16449196-a-c42d4-fad473b,"{'last_updated': {'sender': 1729376581.527151,..."
1,2024-10-19T18:23:01.525797,AssistantResponse,assistant,"{'assistant_response': {'selected': [], 'title...",lnf58395-e2187-2-6fc7819921b0-6566158173b4,lnfe7143be66ee817-16449196-a-c42d4-fad473b,user,{'model_response': {'id': 'chatcmpl-AKC8YD2QqX...


In [6]:
async def mock_search(query: str):
    """
    A mock search function that returns a string
    """
    return f"Search results for: {query}"


res = await branch.operate(
    instruction="Use search tool to gather information on the topic,",
    guidance="Provide a summary of the information",
    context=["What is the meaning of life?"],
    reason=True,
    actions=True,
    tools=[mock_search],
    clear_messages=True,
)

In [7]:
print(as_readable_json(res))

{
    "selected": [],
    "title": "Exploring the Meaning of Life",
    "content": "The task involves gathering information on the philosophical question of the meaning of life. This question has been explored by various philosophers, theologians, and thinkers throughout history, with diverse interpretations and answers. Some view it through the lens of religion, others through existential or scientific perspectives. The task requires using a search tool to gather insights and summarize the findings.",
    "reason": {
        "title": "Initial Exploration",
        "content": "The task is to explore the meaning of life, a complex and multifaceted question. The approach involves using a search tool to gather diverse perspectives and summarize them.",
        "confidence_score": 0.9
    },
    "action_responses": [
        {
            "function": "mock_search",
            "arguments": {
                "query": "meaning of life"
            },
            "output": "Search results for

In [8]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-19T18:23:01.549182,Instruction,user,{'guidance': 'Provide a summary of the informa...,ln963f0ec8-26-e7f1-dd85c7-bc59f1d205e4e4b3,user,lnfe7143be66ee817-16449196-a-c42d4-fad473b,"{'last_updated': {'sender': 1729376583.557962,..."
1,2024-10-19T18:23:03.553584,AssistantResponse,assistant,"{'assistant_response': {'selected': [], 'title...",ln8744e93dd-2b6f-9a8a7-7cd5-1b2348de28586e,lnfe7143be66ee817-16449196-a-c42d4-fad473b,user,{'model_response': {'id': 'chatcmpl-AKC8bi7ezx...
2,2024-10-19T18:23:03.558131,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln83f28317e0a-ae-fa10a08c3-20-9019299303cf,lnfe7143be66ee817-16449196-a-c42d4-fad473b,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,"{'last_updated': {'sender': 1729376583.558372,..."
3,2024-10-19T18:23:03.558259,ActionResponse,assistant,{'action_request_id': 'ln83f28317e0a-ae-fa10a0...,ln12e51b6-6fb-08d602-f-7590c2d08da0dc16238,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,lnfe7143be66ee817-16449196-a-c42d4-fad473b,{'last_updated': {'content': 1729376583.558433...


In [9]:
from IPython.display import Markdown

Markdown(branch.messages[0].chat_msg['content'])


---
 # Task
## **Task guidance**
Provide a summary of the information


## **Task instruction**
Use search tool to gather information on the topic,


## **Task context**
What is the meaning of life?
- respond_schema_info: {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Specify the name of the function to execute. **Choose from the provided `tool_schemas`; do not invent function names.**', 'examples': ['print', 'add', 'len'], 'title': 'Function'}, 'arguments': {'default': {}, 'description': 'Provide the arguments to pass to the function as a dictionary. **Use argument names and types as specified in the `tool_schemas`; do not invent argument names.**', 'examples': [{'num1': 1, 'num2': 2}, {'x': 'hello', 'y': 'world'}], 'title': 'Arguments', 'type': 'object'}}, 'title': 'ActionRequestModel', 'type': 'object'}, 'ReasonModel': {'properties': {'title': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Title'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Content'}, 'confidence_score': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'default': None, 'description': 'Provide an objective numeric confidence score between 0 and 1 (with 3 decimal places) indicating how likely you successfully achieved the task according to user expectation. Interpret the score as:\n- **1**: Very confident in a good job.\n- **0**: Not confident at all.\n- **[0.8, 1]**: You can continue the path of reasoning if needed.\n- **[0.5, 0.8)**: Recheck your reasoning and consider reverting to a previous, more confident reasoning path.\n- **[0, 0.5)**: Stop because the reasoning is starting to be off track.', 'examples': [0.821, 0.257, 0.923, 0.439], 'title': 'Confidence Score'}}, 'title': 'ReasonModel', 'type': 'object'}}, 'required': ['title', 'content'], 'title': 'SelectionModelStepRequest', 'type': 'object'} 

- tool_schemas: {'tools': [{'type': 'function', 'function': {'name': 'mock_search', 'description': 'A mock search function that returns a string', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': None}}, 'required': ['query']}}}]} 



## **Task response format**
**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS.** 
```json
{'selected': {'default': [], 'description': 'select from [positive, negative, neutral]', 'items': {}, 'title': 'Selected', 'type': 'array'}, 'title': {'description': 'Provide a concise title summarizing the step.', 'title': 'Title', 'type': 'string'}, 'content': {'description': 'Describe the content of the step in detail.', 'title': 'Content', 'type': 'string'}, 'reason': {'anyOf': [{'$ref': '#/$defs/ReasonModel'}, {'type': 'null'}], 'default': None, 'description': '**a concise reasoning for the step**', 'title': 'Reason'}, 'action_requests': {'default': [], 'description': 'List of actions to be performed if `action_required` is **True**. Leave empty if no action is required. **When providing actions, you must choose from the provided `tool_schemas`. Do not invent function or argument names.**', 'items': {'$ref': '#/$defs/ActionRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Specify whether the step requires actions to be performed. If **True**, the actions in `action_requests` must be performed. If **False**, the actions in `action_requests` are optional. If no tool_schemas are provided, this field is ignored.', 'title': 'Action Required', 'type': 'boolean'}}
```




---


In [10]:
res = await branch.chat(
    "provide 5 function calling query",
    tools=[mock_search],
    clear_messages=True,
    invoke_action=True,
    imodel=gpt4o,
)

In [11]:
print(res)

Here are five function calling queries using the specified format:

1. 
```json
{'function': 'mock_search', 'arguments': {'query': 'latest technology trends'}}
```

2. 
```json
{'function': 'mock_search', 'arguments': {'query': 'best practices in software development'}}
```

3. 
```json
{'function': 'mock_search', 'arguments': {'query': 'top programming languages in 2023'}}
```

4. 
```json
{'function': 'mock_search', 'arguments': {'query': 'impact of AI on job market'}}
```

5. 
```json
{'function': 'mock_search', 'arguments': {'query': 'future of renewable energy'}}
```


In [12]:
branch.to_df()

,timestamp,lion_class,role,content,ln_id,sender,recipient,metadata
0,2024-10-19T18:23:03.582229,Instruction,user,{'instruction': 'provide 5 function calling qu...,ln2595d3d6-519b-ed92d9-b5a35de6-7ae795f054,user,lnfe7143be66ee817-16449196-a-c42d4-fad473b,"{'last_updated': {'sender': 1729376584.932844,..."
1,2024-10-19T18:23:04.932711,AssistantResponse,assistant,{'assistant_response': 'Here are five function...,ln35f2aa58b9-889-7dfddf-4ca5b3365-0898fdab,lnfe7143be66ee817-16449196-a-c42d4-fad473b,user,{'model_response': {'id': 'chatcmpl-AKC8dtY389...
2,2024-10-19T18:23:04.933964,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln0c09f0b79ba-35-ed0663e-a647a1d81-751e102,lnfe7143be66ee817-16449196-a-c42d4-fad473b,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,"{'last_updated': {'sender': 1729376584.934068,..."
3,2024-10-19T18:23:04.934015,ActionResponse,assistant,{'action_request_id': 'ln0c09f0b79ba-35-ed0663...,lnc94e29803-839c22f5d3-db7cf-e9b-415df8a6b,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,lnfe7143be66ee817-16449196-a-c42d4-fad473b,{'last_updated': {'content': 1729376584.934102...
4,2024-10-19T18:23:04.934134,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln6f33f7514b88937262-a2-3f27-2f-a04311eb6a,lnfe7143be66ee817-16449196-a-c42d4-fad473b,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,"{'last_updated': {'sender': 1729376584.93423, ..."
5,2024-10-19T18:23:04.934172,ActionResponse,assistant,{'action_request_id': 'ln6f33f7514b88937262-a2...,ln16ab963-d68-e336e47-30688-7c9db279ba664e,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,lnfe7143be66ee817-16449196-a-c42d4-fad473b,{'last_updated': {'content': 1729376584.934265...
6,2024-10-19T18:23:04.934294,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",ln6ba01c5d1d3-30-3d8188509-6da-9bbb2714cf0,lnfe7143be66ee817-16449196-a-c42d4-fad473b,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,"{'last_updated': {'sender': 1729376584.934366,..."
7,2024-10-19T18:23:04.934326,ActionResponse,assistant,{'action_request_id': 'ln6ba01c5d1d3-30-3d8188...,lnff00c4ebdb9e7c75b-3-7ab23-ecb-154fff59f8,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,lnfe7143be66ee817-16449196-a-c42d4-fad473b,{'last_updated': {'content': 1729376584.934395...
8,2024-10-19T18:23:04.934423,ActionRequest,assistant,"{'action_request': {'function': 'mock_search',...",lnaab57a4-df18baf-f3-adfe7c26ba19f-996b708,lnfe7143be66ee817-16449196-a-c42d4-fad473b,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,"{'last_updated': {'sender': 1729376584.934493,..."
9,2024-10-19T18:23:04.934453,ActionResponse,assistant,{'action_request_id': 'lnaab57a4-df18baf-f3-ad...,ln5157c62-9af605d66ae7f2-2-e7a1-94e290c8d0,lnbb30ee7b270cd-1b7-bf951-0d-14101315660f5,lnfe7143be66ee817-16449196-a-c42d4-fad473b,{'last_updated': {'content': 1729376584.934521...


In [13]:
from IPython.display import Markdown, display

Markdown(branch.messages[0].chat_msg['content'])


---
 # Task
## **Task instruction**
provide 5 function calling query


## **Task context**
- tool_schemas: {'tools': [{'type': 'function', 'function': {'name': 'mock_search', 'description': 'A mock search function that returns a string', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': None}}, 'required': ['query']}}}]} 



## **Task guidance**


when providing function arguments, please use the following format:

{'function': ..., 'arguments': {...}}






---


In [14]:
for i in branch.messages:
    d = i.chat_msg['content']
    d = as_readable_json(d) if isinstance(d, dict) else d
    display(Markdown(d))


---
 # Task
## **Task instruction**
provide 5 function calling query


## **Task context**
- tool_schemas: {'tools': [{'type': 'function', 'function': {'name': 'mock_search', 'description': 'A mock search function that returns a string', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': None}}, 'required': ['query']}}}]} 



## **Task guidance**


when providing function arguments, please use the following format:

{'function': ..., 'arguments': {...}}






---


Here are five function calling queries using the specified format:

1. 
```json
{'function': 'mock_search', 'arguments': {'query': 'latest technology trends'}}
```

2. 
```json
{'function': 'mock_search', 'arguments': {'query': 'best practices in software development'}}
```

3. 
```json
{'function': 'mock_search', 'arguments': {'query': 'top programming languages in 2023'}}
```

4. 
```json
{'function': 'mock_search', 'arguments': {'query': 'impact of AI on job market'}}
```

5. 
```json
{'function': 'mock_search', 'arguments': {'query': 'future of renewable energy'}}
```

{
    "function": "mock_search",
    "arguments": {
        "query": "latest technology trends"
    }
}

{
    "function": "mock_search",
    "arguments": {
        "query": "latest technology trends"
    },
    "output": "Search results for: latest technology trends"
}

{
    "function": "mock_search",
    "arguments": {
        "query": "best practices in software development"
    }
}

...

{
    "function": "mock_search",
    "arguments": {
        "query": "top programming languages in 2023"
    },
    "output": "Search results for: top programming languages in 2023"
}

{
    "function": "mock_search",
    "arguments": {
        "query": "impact of AI on job market"
    }
}

{
    "function": "mock_search",
    "arguments": {
        "query": "impact of AI on job market"
    },
    "output": "Search results for: impact of AI on job market"
}

{
    "function": "mock_search",
    "arguments": {
        "query": "future of renewable energy"
    }
}

{
    "function": "mock_search",
    "arguments": {
        "query": "future of renewable energy"
    },
    "output": "Search results for: future of renewable energy"
}